In [1]:
#%run ./lib.py

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, DecimalType, TimestampType, DataType, IntegerType, FloatType
from pyspark.sql.functions import col, lit, trim, substring, concat, udf, upper, initcap
from datetime import datetime
import os

In [3]:
conf = SparkConf() \
    .setAppName("projeto_pbi") \
    .setSparkHome('./spark/home')

LAKE_HOME = os.getenv("LAKE_HOME", "/spark/home")

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

22/11/30 01:53:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/30 01:53:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/30 01:53:25 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/11/30 01:53:25 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/11/30 01:53:25 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/11/30 01:53:25 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


In [4]:
TABLE_NAME = 'dim_geolocation'

def create_sk(spark, df, key_column_name, table_name):
    sk = {}
    sk = df.select(col(key_column_name).alias("key")).rdd.zipWithIndex()
    new_sk = sk.map(lambda row: list(row[0]) + [row[1] + 1])
    new_sk_map = new_sk.collectAsMap()

    sk_schema = \
        StructType(
            [StructField('key', StringType(), True),
            StructField('SK', LongType(), True)]
        )

    sk_frame = spark.createDataFrame(new_sk, sk_schema)
    sk_frame.write.mode('overwrite').csv('{}/dataset/e-commerce/02_surrogate_key/sk_{}.csv'.format(LAKE_HOME, table_name), header=True)

    return new_sk_map

def locate_sk(mapping: dict):
    return udf(lambda x: mapping.get(x), IntegerType())

In [5]:
df = spark.read.csv('{}/dataset/e-commerce/01_extract/geolocation_dataset.csv'.format(LAKE_HOME), header=True)

df_geolocation = \
    df.select(
        col('geolocation_zip_code_prefix').cast(StringType()).alias('ID_ZIP_CODE'),
        col('geolocation_lat').cast(StringType()).alias('GEO_LATITUDE'),
        col('geolocation_lng').cast(StringType()).alias('GEO_LONGITUDE'),
        col('geolocation_city').cast(StringType()).alias('DES_CITY'),
        col('geolocation_state').cast(StringType()).alias('COD_STATE')
    )

sk = create_sk(spark=spark, df=df_geolocation, key_column_name='ID_ZIP_CODE', table_name=TABLE_NAME)

In [6]:
dim_geolocation = \
    df_geolocation \
        .withColumn('SK_GEOLOCATION', locate_sk(sk)(col('ID_ZIP_CODE')))

In [7]:
dim_geolocation = \
    dim_geolocation.select(
        col('SK_GEOLOCATION'),
        col('ID_ZIP_CODE'),
        col('GEO_LATITUDE'),
        col('GEO_LONGITUDE'),
        col('DES_CITY'),
        col('COD_STATE')  
    )

dim_geolocation.write.mode('overwrite').csv('{}/dataset/e-commerce/03_dim/{}.csv'.format(LAKE_HOME, TABLE_NAME), header=True)